# Repo Link
https://github.com/mr-kelsey/fa25-aai520-group6

# Objective
Build an autonomous Investment Research Agent that:
1. Plans its research steps for a given stock symbol.
2. Uses tools dynamically (APIs, datasets, retrieval).
3. Self-reflects to assess the quality of its output.
4. Learns across runs (e.g., keeps brief memories or notes to improve future analyses).

# Code

In [ ]:
"""Import required libraies and load environmentals"""
from dotenv import dotenv_values
from huggingface_hub import login
from pathlib import Path
from smolagents import ToolCallingAgent, TransformersModel
from warnings import filterwarnings
from yaml import safe_load

from tools import performance, risk, sentiment, impact, commander, evaluator

# Gluonts uses an outdated pd.df access method that causes a warning.  We are silencing it here to provide a cleaner output
filterwarnings("ignore")
env = dotenv_values(".env")

In [2]:
"""Set up initial agent"""
base_model = TransformersModel(model_id="HuggingFaceTB/SmolLM2-1.7B-Instruct", device_map="auto", max_new_tokens=4096)

In [3]:
"""Configure agents"""
performance_agent = ToolCallingAgent(
    name="Perforance",
    description="Analyze stock performance, historical growth, and future potential.",
    model=base_model,
    tools=performance.tools,
    )

risk_agent = ToolCallingAgent(
    name="Risk",
    description="Measure risk, volatility, and downside probability.",
    model=base_model,
    tools=risk.tools,
    )

sentiment_agent = ToolCallingAgent(
    name="Sentiment",
    description="Gauge market mood through news and media sentiment.",
    model=base_model,
    tools=sentiment.tools,
    )

impact_agent = ToolCallingAgent(
    name="Impact",
    description="Evaluate sustainability, innovation, and ethical governance.",
    model=base_model,
    tools=impact.tools,
    )

evaluation_agent = ToolCallingAgent(
    name="Evaluator",
    description="Audit, validate, and optimize Commander's final decision.",
    model=base_model,  # probably need a more robust agent here...
    tools=evaluator.tools,
    )

commander_agent = ToolCallingAgent(
    name="Commander",
    description="Synthesize all agent outputs and deliver final classification.",
    model=base_model,
    tools=commander.tools,
    managed_agents=[performance_agent, risk_agent, sentiment_agent, impact_agent, evaluation_agent]
    )

commander_agent.visualize()

ToolCallingAgent | HuggingFaceTB/SmolLM2-1.7B-Instruct
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                ┃ Description                               ┃ Arguments                                 ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ make_reccomendation │ Dummy tool to show required format for    │ performance_score (`number`): The         │
│   │                     │ tool use                                  │ performance score as calculated by        │
│   │                     │                                           │ Performance.                              │
│   │                     │                                           │ risk_score (`number`): The risk score as  │
│   │                     │                                           │ calculated by Risk.                       │
│   │                     │                                           │ sentiment_score (`number`): The sentiment │
│   │                     │                                           │ score as calculated by Sentiment.         │
│   │                     │                                           │ impact_score (`number`): The impact score │
│   │                     │                                           │ as calculated by Impact.                  │
│   │ final_answer        │ Provides a final answer to the given      │ answer (`any`): The final answer to the   │
│   │                     │ problem.                                  │ problem                                   │
│   └─────────────────────┴───────────────────────────────────────────┴───────────────────────────────────────────┘
└── 🤖 Managed agents:
    ├── Perforance | ToolCallingAgent | HuggingFaceTB/SmolLM2-1.7B-Instruct
    │   ├── 📝 Description: Analyze stock performance, historical growth, and future potential.
    │   └── 🛠️ Tools:
    │       ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    │       ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
    │       ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │       │ calculate_performance_score │ Calculate a performance score     │ instrument (`string`): The        │
    │       │                             │ based on the model prediction of  │ financial instrument of interest. │
    │       │                             │ future financial instrument       │                                   │
    │       │                             │ movement.                         │                                   │
    │       │ final_answer                │ Provides a final answer to the    │ answer (`any`): The final answer  │
    │       │                             │ given problem.                    │ to the problem                    │
    │       └─────────────────────────────┴───────────────────────────────────┴───────────────────────────────────┘
    ├── Risk | ToolCallingAgent | HuggingFaceTB/SmolLM2-1.7B-Instruct
    │   ├── 📝 Description: Measure risk, volatility, and downside probability.
    │   └── 🛠️ Tools:
    │       ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    │       ┃ Name         ┃ Description                               ┃ Arguments                                ┃
    │       ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │       │ echo_tool    │ Dummy tool to show required format for    │ dummy (`string`): dummy string to show   │
    │       │              │ tool use                                  │                                          │
    │       │ final_answer │ Provides a final answer to the given

In [ ]:
task1 = "NVIDIA stock (NVDA)"
task2 = "Bitwise Bitcoin ETF (BITB)"

commander_agent.run(f"""Your task is to classify each of the following financial instruments into “BUY”, “HOLD”, or “AVOID”:
1. {task1}
2. {task2}

You have a very capable team below you that you should leverage to accomplish your task.  Here are the profiles of each of your team members:
`Performance`
	Role: Evaluates core metrics — returns, growth rate, EPS, and expense ratios if analyzing ETFs.
	Logic: Calculates performance over 1, 3, and 5 years; compares it to sector averages.
	Output: A performance score between 0 and 1 with 1 being the most performant.
    
`Risk`
	Role: Measures risk — beta value, standard deviation, and recent drawdowns.
	Logic: 
	Output: A risk score between -1 and 1 where 1 means “High Risk", -1 menas High Reward”, and 0 represents a “Stable Performer”.
    
`Sentiment`
	Role: Reads financial news — determines market sentiment.
	Logic: Uses NLP to classify tone of each article into -1 if negative, 0 if neutral, and 1 if positive.
	Output: A sentiment score consisting of the mean value of the sentiment classifications for all articles analyzed.
    
`Impact`
	Role: Determines impact — calculates ESG or AI exposure scoring.
	Logic: 
	Output: An impact score between 0 and 1 where the higher the score, the greater the impact.

`Evaluator`
	Role: Reviews worker logs — ensures final recommendation is logically derived from information retrieved.
    Logic: 
    Output: A binary classification: "PASS" if the logic is sound, "FAIL" if the logic is unsound.
    
An example workflow would be:
1) Task each of your workers with doing their job:
	i) Performance should calculate and return a performance score.
    ii) Risk should calculate and return a risk score.
    iii) Sentiment should calculate and return a sentiment score.
    iv) Impact should calculate and return an impact score.
2) Combine your team member's scores by applying the weighted formula: Final_Score = (0.4*Performance) + (0.2*Risk) + (0.3*Sentiment) + (0.1*Impact).
3) Use the `Final_Score` to classify the stock into “BUY”, “HOLD”, “AVOID”.
4) Send your logs and classification to Evaluator for final sign-off.
5) Output your final recommendation: “BUY”, “HOLD”, or “AVOID”
""")

## Prompt Chaining:
### Ingest News

### Preprocess

### Classify

### Extract

### Summarize

## Routing:

In [ ]:
"""Direct content to the right specialist (e.g., earnings, news, or market analyzers)."""
# Example from Lecture:
	# Pull earnings report
	# Search for CEO interviews
	# ask follow up questions if unclear
	# stops when confident in recommendation

## Evaluator–Optimizer:
### Generate analysis

### Evaluate quality

### Refine using feedback

# Summary

* Agent Design and Workflows
* Agent Functions and Capabilities
* Evaluation and Iteration